# Curriculum Learning Final Project
Note: this notebook is not meant to be run, only to display the code. If you wish to replicate my results, run the scripts provided in the same directory (most importantly `train.py`, `train_curriculum.py`, and `evaluate.py`

## Program Execution
Below we define a somewhat safe environment for executing generated code. We do not fully sandbox execution, but disable most harmful OS capabilities.

In [ ]:
# Basically copied from https://github.com/openai/human-eval/blob/master/human_eval/execution.py

from typing import Optional, Callable, Dict
import ast
import contextlib
import faulthandler
import io
import os
import multiprocessing
import platform
import signal
import tempfile


def semisafe_evaluate(completion: str, key : str, timeout : float):
    """
    Evaluates a code completion with some added safety features. 
    Returns a float if code succesfully executes. Returns a string describing 
    error if code is unsuccesfull. 
    """

    def semisafe_execute():

        with create_tempdir():

            # These system calls are needed when cleaning up tempdir.
            import os
            import shutil
            rmtree = shutil.rmtree
            rmdir = os.rmdir
            chdir = os.chdir

            # Disable functionalities that can make destructive changes to the test.
            reliability_guard()

            # Construct the check program and run it.

            try:
                exec_globals = {}
                with swallow_io():
                    with time_limit(timeout):
                         exec(completion, exec_globals)
                if key in exec_globals.keys(): 
                    result.append(exec_globals[key])
                else: 
                    result.append("answer not computed")
            except TimeoutException:
                result.append("timed out")
            except BaseException as e:
                result.append(f"failed: {e}")

            # Needed for cleaning up.
            shutil.rmtree = rmtree
            os.rmdir = rmdir
            os.chdir = chdir

    manager = multiprocessing.Manager()
    result = manager.list()

    p = multiprocessing.Process(target=semisafe_execute)
    p.start()
    p.join(timeout=timeout + 1)
    if p.is_alive():
        p.kill()

    if not result:
        result.append("timed out")

    return result[0]


@contextlib.contextmanager
def time_limit(seconds: float):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.setitimer(signal.ITIMER_REAL, seconds)
    signal.signal(signal.SIGALRM, signal_handler)
    try:
        yield
    finally:
        signal.setitimer(signal.ITIMER_REAL, 0)


@contextlib.contextmanager
def swallow_io():
    stream = WriteOnlyStringIO()
    with contextlib.redirect_stdout(stream):
        with contextlib.redirect_stderr(stream):
            with redirect_stdin(stream):
                yield


@contextlib.contextmanager
def create_tempdir():
    with tempfile.TemporaryDirectory() as dirname:
        with chdir(dirname):
            yield dirname


class TimeoutException(Exception):
    pass


class WriteOnlyStringIO(io.StringIO):
    """ StringIO that throws an exception when it's read from """

    def read(self, *args, **kwargs):
        raise IOError

    def readline(self, *args, **kwargs):
        raise IOError

    def readlines(self, *args, **kwargs):
        raise IOError

    def readable(self, *args, **kwargs):
        """ Returns True if the IO object can be read. """
        return False


class redirect_stdin(contextlib._RedirectStream):  # type: ignore
    _stream = 'stdin'


@contextlib.contextmanager
def chdir(root):
    if root == ".":
        yield
        return
    cwd = os.getcwd()
    os.chdir(root)
    try:
        yield
    except BaseException as exc:
        raise exc
    finally:
        os.chdir(cwd)


def reliability_guard(maximum_memory_bytes: Optional[int] = None):
    """
    This disables various destructive functions and prevents the generated code
    from interfering with the test (e.g. fork bomb, killing other processes,
    removing filesystem files, etc.)
    WARNING
    This function is NOT a security sandbox. Untrusted code, including, model-
    generated code, should not be blindly executed outside of one. See the 
    Codex paper for more information about OpenAI's code sandbox, and proceed
    with caution.
    """

    if maximum_memory_bytes is not None:
        import resource
        resource.setrlimit(resource.RLIMIT_AS, (maximum_memory_bytes, maximum_memory_bytes))
        resource.setrlimit(resource.RLIMIT_DATA, (maximum_memory_bytes, maximum_memory_bytes))
        if not platform.uname().system == 'Darwin':
            resource.setrlimit(resource.RLIMIT_STACK, (maximum_memory_bytes, maximum_memory_bytes))

    faulthandler.disable()

    import builtins
    builtins.exit = None
    builtins.quit = None

    import os
    os.environ['OMP_NUM_THREADS'] = '1'

    os.kill = None
    os.system = None
    os.putenv = None
    os.remove = None
    os.removedirs = None
    os.rmdir = None
    os.fchdir = None
    os.setuid = None
    os.fork = None
    os.forkpty = None
    os.killpg = None
    os.rename = None
    os.renames = None
    os.truncate = None
    os.replace = None
    os.unlink = None
    os.fchmod = None
    os.fchown = None
    os.chmod = None
    os.chown = None
    os.chroot = None
    os.fchdir = None
    os.lchflags = None
    os.lchmod = None
    os.lchown = None
    os.getcwd = None
    os.chdir = None

    import shutil
    shutil.rmtree = None
    shutil.move = None
    shutil.chown = None

    import subprocess
    subprocess.Popen = None  # type: ignore
    
    # Next line causes an error for some reason. 
    #__builtins__['help'] = None

    import sys
    sys.modules['ipdb'] = None
    sys.modules['joblib'] = None
    sys.modules['resource'] = None
    sys.modules['psutil'] = None
    sys.modules['tkinter'] = None

## Dataset
We use the following function and class to parse the MathQAPython jsons and to retrieve tokenized instances. 

In [ ]:
import torch 
import json 
from pathlib import Path 

def read_mathqapython(path): 
    path = Path(path)
    with open(path, 'rb') as f: 
        mathqapython_list = json.load(f)


    return mathqapython_list

class MathQAPython(torch.utils.data.Dataset): 
    def __init__(self, sorted_instances, tokenizer, max_length): 
        self.data = sorted_instances
        self.tokenizer = tokenizer 
        self.max_length = max_length
    

    def __getitem__(self, idx): 
        instance = self.data[idx]
        text = instance['text'] 
        solution = instance['text'] + '\n' + instance['code'] 
        answer = instance['answer']

        text_encode = self.tokenizer(text, 
                max_length=self.max_length, truncation=True, 
                padding='max_length', return_tensors='pt')
        solution_encode = self.tokenizer(solution, 
                max_length=self.max_length, truncation=True, 
                padding='max_length', return_tensors='pt')
        text_ids = text_encode['input_ids'].squeeze()
        solution_ids = solution_encode['input_ids'].squeeze()
        solution_attn = solution_encode['attention_mask'].squeeze()

        return text_ids.long(), solution_ids.long(), solution_attn.long(), answer


    def __len__(self): 
        return len(self.data) 

## Baseline Model
Our baseline model is straightforward: we simply use the default Huggingface infrastructure for fine-tuning autoregressive language models. 

In [ ]:
import sys 
import torch 
from torch.utils.data import RandomSampler, BatchSampler 
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import TrainingArguments, Trainer 

from dataset import read_mathqapython, MathQAPython

from tqdm import tqdm 

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# Tokenizer 
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token 
max_length = 450

# Loading data
print("loading data")
data = read_mathqapython('data/mathqapython_train.json')
train_set = MathQAPython(data, tokenizer, max_length)

# model 
print('loading model')
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")


training_args = TrainingArguments(output_dir="./train_results/default",
                                  num_train_epochs=5,
                                  per_device_train_batch_size=8,
                                  logging_strategy="epoch",
                                  save_strategy="epoch",
                                  weight_decay=0.01,
                                  warmup_steps = 100,
                                  logging_dir="./train_results/default/log"
                                  )

def data_collator(data):
    return {'input_ids': torch.stack([f[1] for f in data]),
            'attention_mask': torch.stack([f[2] for f in data]),
            'labels': torch.stack([f[1] for f in data])
           }

Trainer(model=model, args=training_args, train_dataset=train_set,
        data_collator=data_collator).train()

## Curriculum Learning
We implement curriculum learning by defining a `CurriculumSampler` that subclasses the PyTorch `Sampler`. The `CurriculumSampler` presupposes that the dataset is sorted according to the curriculum learning scoring function. Then, the `CurriculumSampler`'s job is to tell the dataloader which dataset indices each minibatch should consist of. 

To sample minibatches, we use the same algorithm found in (Hacohen and Weinshall, 2019) (see writeup bibliography for reference). Namely, to form the `i`th minibatch, we sample from the `pacer(i)` easiest instances in the dataset. 

Note the definition of our pacing function is as described in the writeup. 

In [ ]:
import math 
import random 
from torch.utils.data import Sampler 

def pacer(itr, n, step_length, increase, start_prop): 
    return round(n * min(1, start_prop * math.pow(increase, math.floor(itr / step_length))))

def get_instance(num_instances): 
    while True: 
        idxs = list(range(num_instances))
        random.shuffle(idxs)
        for idx in idxs: 
            yield idx


class CurriculumSampler(Sampler): 
    def __init__(self, num_iters, iters_per_refresh, increase, start_prop, 
            batch_size, n): 
        self.num_iters = num_iters
        self.iters_per_refresh = iters_per_refresh
        self.increase = increase
        self.start_prop = start_prop
        self.batch_size = batch_size 
        self.n = n 

    def __iter__(self): 
        num_iters = self.num_iters
        iters_per_refresh = self.iters_per_refresh
        increase = self.increase 
        start_prop = self.start_prop
        batch_size = self.batch_size
        n = self.n 

        for itr in range(num_iters): 
            if itr % iters_per_refresh == 0:
                # Do refresh 
                num_instances = pacer(itr, n, iters_per_refresh, increase, start_prop)
                instance_gen = get_instance(num_instances)
            batch = [next(instance_gen) for _ in range(batch_size)]
            yield batch 

    def __len__(self): 
        return self.num_iters

After defining the `CurriculumSampler`, writing our training loop becomes straightforward. The only differences from the baseline is that we must sort our dataset according to the curriculum scoring function and that subclass the Huggingface `Trainer` in order to construct a Pytorch `Dataloader` using the `CurriculumSampler` instead of the default `Sampler`. 

In [ ]:
import sys 
import random 
import math 
import torch 
from torch.utils.data import DataLoader
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import TrainingArguments, Trainer 

from dataset import read_mathqapython, MathQAPython

from curriculum import CurriculumSampler

from tqdm import tqdm 

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Tokenizer 
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token 
max_length = 450
# Loading data
print("loading data")
data = read_mathqapython('data/mathqapython_train.json')

def seqlen(instance): 
    return len(tokenizer.encode(instance['text']+instance['code']))

sorted_data = sorted(data, key=seqlen)
train_set = MathQAPython(sorted_data, tokenizer, max_length)

# model 
print('loading model')
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# curriculum learning stuff 



class CurriculumTrainer(Trainer): 
    def get_train_dataloader(self) -> DataLoader: 
        sampler = CurriculumSampler(12000, 
                                    1000, 
                                    1.4, 
                                    0.1, 
                                    8, 
                                    len(data)
                                    )

        return DataLoader(
                self.train_dataset, 
                collate_fn = self.data_collator, 
                num_workers = self.args.dataloader_num_workers, 
                pin_memory = self.args.dataloader_pin_memory, 
                batch_sampler = sampler
                )



# Training loop
training_args = TrainingArguments(output_dir="./train_results/curriculum2",
                                  num_train_epochs=1,
                                  logging_steps = 500, 
                                  save_steps = 500,
                                  weight_decay=0.01,
                                  warmup_steps = 100,
                                  logging_dir="./train_results/curriculum2/log"
                                  )

def data_collator(data):
    return {'input_ids': torch.stack([f[1] for f in data]),
            'attention_mask': torch.stack([f[2] for f in data]),
            'labels': torch.stack([f[1] for f in data])
           }

CurriculumTrainer(model=model, args=training_args, train_dataset=train_set,
        data_collator=data_collator).train()

## Evaluation

Our evaluation script is a standard-one for the text-to-code task that evaluates pass@1 and pass@10.

In [ ]:
import sys
import re
import random
import yaml
import json
import os
import itertools
from tqdm import tqdm 
random.seed(1)

import numpy as np 
np.random.seed(1)

import torch
from torch.utils.data import DataLoader
torch.manual_seed(1)

from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from dataset import read_mathqapython

from execution import semisafe_evaluate

device = 'cuda:1'

model_path = sys.argv[1]
outfile = sys.argv[2]

# Loads model and data 
print('loading model and data...')
model = GPTNeoForCausalLM.from_pretrained(model_path).to(device)

# Test set
data = read_mathqapython('data/mathqapython_test.json')
random.shuffle(data)
data = data[:1000]

# tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token


def pass_k(lst, k): 
    """
    lst: Boolean list 
    k: value of pass@k to calculate. 
    """
    n = len(lst)
    c = sum(lst)
    if n - c < k: return 1.0 
    return 1.0 - np.prod(1.0 - k / 
                        np.arange(n-c+1, n+1))


results = []
for instance in tqdm(data): 
    label = instance['answer']
    encoded_prompt = tokenizer.encode(instance['text'], 
            return_tensors='pt').to(device)
    prompt_length = torch.numel(encoded_prompt)
    with torch.no_grad(): 
        out = model.generate(
                input_ids=encoded_prompt,
                do_sample=True ,
                temperature=0.2, 
                max_length = min([prompt_length+250, 450]), 
                pad_token_id=tokenizer.eos_token_id, 
                num_return_sequences=10
                )
        generated_ids = [ids[prompt_length:] for ids in out]

    untrunced_bodies = [tokenizer.decode(sample, skip_special_tokens=True)
            for sample in generated_ids]
    
    re_key = '^answer.*?\n'
    bodies = [completion[:re.search(re_key, completion).span()[1]]
            if re.search(re_key, completion) else completion 
            for completion in untrunced_bodies]

    answers = [semisafe_evaluate(program, 'answer', 1) for program in bodies]

    passed_lst = [(abs((answer - label)/label) < 0.01)
            if isinstance(answer, float) else False for answer in answers]

    result = dict()

    result[10] = pass_k(passed_lst, 10)
    result[1] = pass_k(passed_lst, 1)

    if True in passed_lst: 
        best_completion = bodies[passed_lst.index(True)]
    else: 
        best_completion = bodies[0]

    result['best_completion'] = best_completion 

    results.append(result)

pass10scores = [instance[10] for instance in results]
pass10average = sum(pass10scores)/len(pass10scores)
print("pass 10: ", pass10average)

pass1scores = [instance[1] for instance in results]
pass1average = sum(pass1scores)/len(pass1scores)
print("pass 1: ", pass1average)

to_dump = {'pass1': pass1average, 
        'pass10': pass10average, 
        'results': results}

with open(outfile, 'w') as fle: 
    json.dump(to_dump, fle)